In [ ]:
# ============================================
# 🌎 Air Quality Forecast App – Los Angeles
# Integrating WAQI (Real-time) & Meteomatics (Historical + 24hr API Forecast)
# Using a 24hr SARIMA model for forecasting
# ============================================

# Ensure necessary libraries are installed
# !pip install requests pandas plotly scikit-learn statsmodels

import requests
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings

# Suppress warnings from statsmodels to keep the output clean
warnings.filterwarnings("ignore")

# ==============================
# 🔐 User Config
# ==============================
CITY = "Los Angeles"
LAT, LON = 34.0522, -118.2437  # Coordinates for Los Angeles

# --- API TOKENS & CREDENTIALS ---
# Get a free token from https://aqicn.org/data-platform/token/
AQICN_API_TOKEN = "adab0250428e88549d94621afbdaa2c1c36d00c5"

# Enter your Meteomatics API credentials here
METEO_USER = "menon_prithvishankar"
METEO_PASS = "Q6bRXp5GWf1mLoLa7Jz1"

# ==============================
# 1️⃣ Fetch Real-time Data (Air Quality Open Data Platform)
# ==============================
def get_waqi_data(lat, lon, token):
    """
    Fetches the latest air quality data from the World Air Quality Index project.
    """
    url = f"https://api.waqi.info/feed/geo:{lat};{lon}/?token={token}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get("status") == "ok":
            return data["data"].get("iaqi", {})
        else:
            print(f"❌ WAQI API returned an error: {data.get('data')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching data from WAQI API: {e}")
        return None

# ==============================
# 2️⃣ Fetch Historical & API Forecast Data (Meteomatics)
# ==============================
def get_air_quality_data_meteomatics(lat, lon, pollutants, username, password):
    """
    Fetches historical (90 days) and forecast (24 hours) air quality data
    from the Meteomatics API using Basic Authentication.
    """
    # Mapping from WAQI names to Meteomatics parameter names
    pollutant_mapping = {
        'pm25': 'pm2p5:ugm3',
        'pm10': 'pm10:ugm3',
        'o3': 'o3:ugm3',
        'no2': 'no2:ugm3',
        'so2': 'so2:ugm3',
        'co': 'co:ugm3'
    }
    # Create a reverse mapping to rename columns back to the simple format
    reverse_mapping = {v: k for k, v in pollutant_mapping.items()}

    # Filter for pollutants available in Meteomatics and build the parameter string
    meteomatics_params = [pollutant_mapping[p] for p in pollutants if p in pollutant_mapping]
    if not meteomatics_params:
        print("❌ None of the available real-time pollutants can be mapped for Meteomatics data.")
        return pd.DataFrame()
    params_str = ",".join(meteomatics_params)

    # Define the time range (90 days past to 24 hours future, hourly interval)
    end_time = dt.datetime.utcnow() + dt.timedelta(hours=24) # <-- CHANGED
    start_time = dt.datetime.utcnow() - dt.timedelta(days=90)
    time_str = f"{start_time.isoformat(timespec='seconds')}Z--{end_time.isoformat(timespec='seconds')}Z:PT1H"

    # Location string
    location_str = f"{lat},{lon}"

    # Construct the final API URL
    url = f"https://api.meteomatics.com/{time_str}/{params_str}/{location_str}/json"
    print("Querying Meteomatics API for 24-hour forecast...")

    try:
        # Make the request with authentication
        response = requests.get(url, auth=(username, password))
        response.raise_for_status()  # This will raise an error for bad responses (4xx or 5xx)
        data = response.json()

        if data.get('status') != 'OK' or not data.get('data'):
            print(f"❌ Meteomatics API returned an error: {data.get('message')}")
            return pd.DataFrame()

        # Parse the Meteomatics JSON structure into a pandas DataFrame
        all_data = {}
        for param_data in data['data']:
            param_name = param_data['parameter']
            dates = param_data['coordinates'][0]['dates']
            values = [d['value'] for d in dates]
            all_data[param_name] = values
        
        df = pd.DataFrame(all_data)
        # The 'date' is the same for all parameters, so we can take it from the first one
        df['datetime'] = [d['date'] for d in data['data'][0]['coordinates'][0]['dates']]
        df['datetime'] = pd.to_datetime(df['datetime'], utc=True)

        # Rename columns to the simple format for consistency (e.g., 'pm25')
        df.rename(columns=reverse_mapping, inplace=True)
        
        print(f"✅ Fetched historical & API forecast data from Meteomatics. Size: {df.shape}")
        return df

    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error fetching from Meteomatics: {e.response.status_code} {e.response.text}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ An unexpected error occurred with Meteomatics API: {e}")
        return pd.DataFrame()


# ==============================
# 3️⃣ Advanced Forecast (SARIMA Model)
# ==============================
def forecast_pollutant_sarima(df, pollutant, hours_ahead=24):
    """
    Generates a forecast for a given pollutant using a SARIMA model.
    This should be trained ONLY on historical data.
    """
    df_filtered = df.dropna(subset=[pollutant]).copy()
    if len(df_filtered) < 50:
        print(f"Not enough historical data to forecast {pollutant} with SARIMA.")
        return None

    time_series = df_filtered.set_index('datetime')[pollutant].asfreq('H')
    order = (1, 1, 1)
    seasonal_order = (1, 1, 1, 24)

    try:
        model = ARIMA(time_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit()
        forecast = model_fit.get_forecast(steps=hours_ahead)
        forecast_df = forecast.summary_frame()
        future_time = [time_series.index[-1] + dt.timedelta(hours=i+1) for i in range(hours_ahead)]

        return pd.DataFrame({
            "datetime": future_time,
            f"{pollutant}_forecast": forecast_df['mean'].values,
            f"{pollutant}_lower_ci": forecast_df['mean_ci_lower'].values,
            f"{pollutant}_upper_ci": forecast_df['mean_ci_upper'].values
        })
    except Exception as e:
        print(f"❌ SARIMA model failed for {pollutant}: {e}")
        return None

# ==============================
# 4️⃣ Visualization (Plotly with All Data Series)
# ==============================
def plot_pollutant(historical_df, api_forecast_df, sarima_forecast_df, pollutant, city):
    """
    Plots historical data, the Meteomatics API forecast, and our SARIMA forecast.
    """
    pollutant_name = pollutant.upper()
    title = f"{pollutant_name} Data & 24-Hour Forecast Comparison – {city}"
    fig = go.Figure()

    # 1. Historical Data (Solid Blue)
    fig.add_trace(go.Scatter(
        x=historical_df["datetime"], y=historical_df[pollutant],
        mode='lines', name='Historical Data', line=dict(color='blue')
    ))

    # 2. API Forecast (Dashed Sky Blue)
    fig.add_trace(go.Scatter(
        x=api_forecast_df["datetime"], y=api_forecast_df[pollutant],
        mode='lines', name='API Forecast', line=dict(color='#87CEEB', dash='dash')
    ))

    # 3. Our SARIMA Forecast (Dotted Red)
    if sarima_forecast_df is not None:
        forecast_col = f"{pollutant}_forecast"
        lower_ci_col = f"{pollutant}_lower_ci"
        upper_ci_col = f"{pollutant}_upper_ci"

        fig.add_trace(go.Scatter(
            x=sarima_forecast_df["datetime"], y=sarima_forecast_df[forecast_col],
            mode='lines', name='Our SARIMA Forecast', line=dict(color='red', dash='dot')
        ))

        # Confidence Interval for SARIMA forecast (Light Red Fill)
        fig.add_trace(go.Scatter(
            x=pd.concat([sarima_forecast_df["datetime"], sarima_forecast_df["datetime"][::-1]]),
            y=pd.concat([sarima_forecast_df[upper_ci_col], sarima_forecast_df[lower_ci_col][::-1]]),
            fill='toself', fillcolor='rgba(255, 0, 0, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip", showlegend=True, name='SARIMA 95% Confidence'
        ))
        
        # Define zoom ranges
        zoom_start = api_forecast_df["datetime"].iloc[0] - dt.timedelta(hours=24) if not api_forecast_df.empty else historical_df["datetime"].iloc[-1]
        zoom_end = sarima_forecast_df["datetime"].iloc[-1] + dt.timedelta(hours=1)
        full_view_start = historical_df["datetime"].min()
        full_view_end = sarima_forecast_df["datetime"].max()
        
    else: # Fallback if SARIMA fails
        full_view_start = historical_df["datetime"].min()
        full_view_end = api_forecast_df["datetime"].max() if not api_forecast_df.empty else historical_df["datetime"].max()
        zoom_start = api_forecast_df["datetime"].iloc[0] - dt.timedelta(hours=24) if not api_forecast_df.empty else historical_df["datetime"].iloc[-1]
        zoom_end = api_forecast_df["datetime"].max() if not api_forecast_df.empty else historical_df["datetime"].max()
        

    fig.update_layout(
        title=title,
        xaxis_title="Time (UTC)",
        yaxis_title=f"{pollutant_name} (Concentration / AQI)",
        legend_title_text='',
        updatemenus=[dict(
            type="buttons", direction="right",
            x=0.57, xanchor="left", y=1.15, yanchor="top",
            showactive=True,
            buttons=list([
                dict(label="Full View", method="relayout",
                     args=[{"xaxis.range": [full_view_start, full_view_end]}]),
                dict(label="Zoom to Forecast", method="relayout",
                     args=[{"xaxis.range": [zoom_start, zoom_end]}])
            ])
        )]
    )
    fig.show()

# ==============================
# 5️⃣ Health Alerts
# ==============================
def generate_alerts(df, pollutant, threshold):
    """
    Checks if a pollutant's forecasted value exceeds a given threshold.
    Based on our SARIMA model's forecast.
    """
    pollutant_name = pollutant.upper()
    forecast_col = f"{pollutant}_forecast"
    high_values = df[df[forecast_col] > threshold]
    if not high_values.empty:
        max_val = high_values[forecast_col].max()
        print(f"🚨 ALERT! Unhealthy {pollutant_name} levels predicted by SARIMA. Max forecast: {max_val:.2f} (Threshold: {threshold})")
    else:
        print(f"✅ SARIMA forecasted {pollutant_name} levels are within safe limits.")


# ==============================
# 6️⃣ Main Analysis Loop
# ==============================
real_time_aq_data = get_waqi_data(LAT, LON, AQICN_API_TOKEN)
if real_time_aq_data:
    print(f"✅ Real-time data fetched for {CITY}: {list(real_time_aq_data.keys())}")
    available_pollutants_realtime = list(real_time_aq_data.keys())
    
    # Fetch combined historical and forecast data from the Meteomatics API
    combined_df = get_air_quality_data_meteomatics(
        LAT, LON, available_pollutants_realtime, METEO_USER, METEO_PASS
    )

    if not combined_df.empty:
        # Define the split point (current time)
        now_utc = pd.Timestamp.utcnow()
        historical_df = combined_df[combined_df['datetime'] <= now_utc].copy()
        api_forecast_df = combined_df[combined_df['datetime'] > now_utc].copy()
        
        print(f"Data split: {len(historical_df)} historical points, {len(api_forecast_df)} API forecast points.")

        HEALTH_THRESHOLDS = {'pm25': 35, 'pm10': 75, 'o3': 70, 'no2': 100, 'so2': 75, 'co': 9000}
        available_pollutants = [col for col in combined_df.columns if col not in ['datetime']]

        for pollutant in available_pollutants:
            print(f"\n--- Analyzing {pollutant.upper()} ---")

            # 1. Generate SARIMA Forecast using ONLY historical data
            sarima_forecast_df = forecast_pollutant_sarima(historical_df, pollutant, hours_ahead=24) # <-- CHANGED

            # 2. Plot all three data series
            plot_pollutant(historical_df, api_forecast_df, sarima_forecast_df, pollutant, CITY)

            # 3. Check for Health Alerts based on our SARIMA forecast
            if sarima_forecast_df is not None:
                threshold = HEALTH_THRESHOLDS.get(pollutant, 150)
                generate_alerts(sarima_forecast_df, pollutant, threshold)
else:
    print("Could not fetch real-time data to proceed with analysis.")